In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.database.tables import Base as BuildingMotif_tables_base

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("my-building")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
Library.load(ontology_graph="../libraries/brick/Brick-subset.ttl")

Library(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f7d52e55e50>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = Library.load(directory="../libraries/ashrae/guideline36/")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'zone', 'occ', 'ztemp', 'co2', 'dat', 'dmp'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b00b4da0> ;
    brick:hasPart <urn:my-building/dmp_2cd642e1> ;
    brick:hasPoint <urn:my-building/co2_18dc64b3>,
        <urn:my-building/dat_6a9218c8>,
        <urn:my-building/occ_c0323891>,
        <urn:my-building/ztemp_e779bba4> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_906cf117> ;
    brick:hasPart <urn:my-building/dmp_24876470> ;
    brick:hasPoint <urn:my-building/co2_35b061da>,
        <urn:my-building/dat_6a6ccc44>,
        <urn:my-building/occ_142aa689>,
        <urn:my-building/ztemp_0e7856e2> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_78698f09> ;
    brick:hasPart <urn:my-building/dmp_b2aef288> ;
    brick:hasPoint <urn:my-building/co2_9b5e694c>,
        <urn:my-building/dat_b09b8fa5>,
        <urn:my-building/occ_00bde80d>,
        <urn:my-building/ztemp_9d9e630b> .



In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_3784a4d6> a brick:VAV ;
    brick:feeds <urn:my-building/zone_a34e2294> ;
    brick:hasPart <urn:my-building/dmp_c71c52de> ;
    brick:hasPoint <urn:my-building/co2_06084adb>,
        <urn:my-building/dat_f6efcf10>,
        <urn:my-building/occ_cfd91e84>,
        <urn:my-building/ztemp_c47f0035> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_eccc2a25> a brick:VAV ;
    brick:feeds <urn:my-building/zone_f1f0ab32> ;
    brick:hasPart <urn:my-building/dmp_8444550d> ;
    brick:hasPoint <urn:my-building/co2_d8656ec2>,
        <urn:my-building/dat_73507946>,
        <urn:my-building/occ_a6c2fc81>,
        <urn:my-building/ztemp_bc59ffaf> .

<urn:my-building/co2_d8656ec2> a brick:CO2_Level_Sensor .

<urn:my-building/dat_73507946> a brick:Discharge_Air_Temperature_Sensor .

<urn:my-building/dmp_8444550d> a brick:Damper ;
    brick:hasPoint <urn:my-building/dmppos_71ca02ae> .

<urn:my-building/dmppos_71ca02ae> a brick:Damper_Position_Command .

<urn:my-building/occ_a6c2fc81> a brick:Occupancy_Sensor .

<urn:my-building/zone_f1f0ab32> a brick:HVAC_Zone .

<urn:my-building/ztemp_bc59ffaf> a brick:Zone_Air_Temperature_Sensor .




In [14]:
print(inlined.body.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:VAV ;
    brick:feeds <urn:___param___#zone> ;
    brick:hasPart <urn:___param___#dmp> ;
    brick:hasPoint <urn:___param___#co2>,
        <urn:___param___#dat>,
        <urn:___param___#occ>,
        <urn:___param___#ztemp> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos> .

<urn:___param___#dmppos> a brick:Damper_Position_Command .

<urn:___param___#occ> a brick:Occupancy_Sensor .

<urn:___param___#zone> a brick:HVAC_Zone .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .




Run another example w/ chiller plant

In [15]:
lib = Library.load(directory="../libraries/chiller-plant/")

In [16]:
chiller = lib.get_template_by_name("chiller")
inlined = chiller.inline_dependencies()
print(inlined.body.serialize())
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<urn:___param___#name> a brick:Chiller ;
    brick:hasPart <urn:___param___#chwp>,
        <urn:___param___#cnd>,
        <urn:___param___#cvlv> ;
    brick:hasPoint <urn:___param___#chwrt>,
        <urn:___param___#chwst> ;
    brick:isMeteredBy <urn:___param___#meter> .

<urn:___param___#bypass_valve-vlv_cmd-122bcafd> a brick:Valve_Command .

<urn:___param___#bypass_valve-vlv_cmd-d9542ec6> a brick:Valve_Command .

<urn:___param___#chwp> a brick:Chilled_Water_Pump,
        brick:Pump ;
    brick:hasPart <urn:___param___#pump-bypass_valve-51ffe1de>,
        <urn:___param___#pump-control_valve-51ffe1de> ;
    brick:hasPoint <urn:___param___#pump-mode-51ffe1de>,
        <urn:___param___#pump-run-51ffe1de>,
        <urn:___param___#pump-vsd-51ffe1de> ;
    brick:isMeteredBy <urn:___param___#pump-meter-51ffe1de> .

<urn:___param___#chwrt> a brick:Chilled_Water_Return_Temperatur